# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Estimating-Euler-Equation-with-Generalized-Method-of-Moments-with-statsmodels" data-toc-modified-id="Estimating-Euler-Equation-with-Generalized-Method-of-Moments-with-statsmodels-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Estimating Euler Equation with Generalized Method of Moments with statsmodels</a></div><div class="lev2 toc-item"><a href="#Setup-and-Data" data-toc-modified-id="Setup-and-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup and Data</a></div><div class="lev2 toc-item"><a href="#The-moment-equations-and-GMM---version-1" data-toc-modified-id="The-moment-equations-and-GMM---version-1-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The moment equations and GMM - version 1</a></div><div class="lev2 toc-item"><a href="#The-moment-equations-and-GMM---version-2" data-toc-modified-id="The-moment-equations-and-GMM---version-2-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>The moment equations and GMM - version 2</a></div><div class="lev2 toc-item"><a href="#Compare" data-toc-modified-id="Compare-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Compare</a></div><div class="lev2 toc-item"><a href="#Extra:-Iterated-GMM" data-toc-modified-id="Extra:-Iterated-GMM-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Extra: Iterated GMM</a></div><div class="lev2 toc-item"><a href="#Extra:-Problems-with-Optimization" data-toc-modified-id="Extra:-Problems-with-Optimization-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Extra: Problems with Optimization</a></div>

# Estimating Euler Equation with Generalized Method of Moments with statsmodels

One of the earliest application of GMM is the estimation of Euler equations in rational expectation models 
(Hansen and Singleton 1982). We can read about it in many textbooks, but here I just want to present it as 
another test case for GMM in statsmodels.

The short form is that optimization by economic agents with rational expectation implies the following orthogonality condition in expectation:

$\hspace{10pt} E [z_t \hspace{3pt} (1 - \beta  (1 + r_{t+1})  (c_{t+1} / c_t)^{-\gamma}) ]$
    

where beta is the discount factor of the agent, and gamma is the coefficient reflecting constant relative risk aversion. R is a rate of return on assets or interest rate, c is consumption
(I have not looked at this part of economics in a long time.)

The main point is that we cannot treat current and future consumption and future interest rates as exogenous when we try to estimate the parameters, since economic agents have more information about those variables than the econometrician. Those variables will be correlated with any residuals in the estimation and give us inconsistent estimates of the parameters if we don't take the endogeneity into account.


## Setup and Data

In [1]:

"""

Created on Tue Oct 08 09:27:31 2013

Author: Josef Perktold
"""

import numpy as np
import pandas as pd

from statsmodels.sandbox.regression import gmm

infile = 'https://gist.githubusercontent.com/josef-pkt/8128539/raw/7b34d435fae7a49a469085a71d8aebb30f0ba64b/consumption.csv'

# dta = pd.read_csv('consumption.csv', parse_dates=[0])
dta = pd.read_csv(infile, parse_dates=[0])

dta.iloc[:5]

/Users/poudel/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,qtr,r,c
0,1947-01-01,0.0038,1017.2
1,1947-04-01,0.0038,1034.0
2,1947-07-01,0.0066,1037.5
3,1947-10-01,0.0085,1037.7
4,1948-01-01,0.0097,1042.6


Next, I create the lagged and leading variables for the estimation. 
As instruments we use lagged interest rate and current and lagged consumption growth.

In [2]:
dta['c_growth'] = dta['c'] / dta['c'].shift(1)
dta['c_growth_lag1'] = dta['c_growth'].shift(1)
dta['r_lag1'] = dta['r'].shift(1)
dta['r_lag2'] = dta['r'].shift(2)
dta['r_forw1'] = dta['r'].shift(-1)
dta['c_lag1'] = dta['c'].shift(1)
dta['c_forw1'] = dta['c'].shift(-1)
dta['const'] = 1

dta_clean = dta.dropna()


endog_df = dta_clean[['r_forw1', 'c_forw1', 'c']]
exog_df = endog_df
instrument_df = dta_clean[['r_lag1', 'r_lag2', 'c_growth', 'c_growth_lag1',
                           'const']]

endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])

## The moment equations and GMM - version 1

Currently statsmodels has two ways of specifying moment conditions. 
The first uses general non-linear functions for the (unconditional) moment condition:
    
$\hspace{10pt} E[m(params)] = 0$

The second version uses an instrumental variables approach with additive error structure
    
\begin{equation*} \hspace{10pt} E[z \hspace{3pt} (y - f(x, params)] = 0 \end{equation*}

where z are the instruments and $u = (y - f(x, params)$ defines an additive residual, and `x` 
are explanatory variables that can be endogenous or exogenous.
                                  
In the following I use the class `NonlinearIVGMM`, which implements the second form of the moment conditions.
However, our Euler equation doesn't fit's this only if we use endog or y as a constant.
I'm not showing an example how we can use the first generic form, class `GMM`, nor how
we could define the moment conditions directly by subclassing.

**The first version**

In the first version, we use `endog = 0` and define `u` to be the the Euler equation without the instrument part.

In [3]:
def moment_consumption1(params, exog):
    beta, gamma = params
    r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)
    
    # moment condition without instrument    
    err = 1 - beta * (1 + r_forw1) * np.power(c_forw1 / c, -gamma)
    return -err

In [4]:
endog1 = np.zeros(exog.shape[0])    
mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4)
w0inv = np.dot(instrument.T, instrument) / len(endog1)
res1 = mod1.fit([1,-1], maxiter=2, inv_weights=w0inv) 

Optimization terminated successfully.
         Current function value: 0.000539
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.152764
         Iterations: 6
         Function evaluations: 11
         Gradient evaluations: 11


In [5]:
print(res1.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        36.51
Model:                 NonlinearIVGMM   Prob (Hansen J):              1.18e-08
Method:                           GMM                                         
Date:                Mon, 24 Jun 2019                                         
Time:                        12:14:24                                         
No. Observations:                 239                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
discount       0.8977      0.017     52.839      0.000       0.864       0.931
CRRA          -6.7989      2.051     -3.316      0.001     -10.818      -2.780


However, the default GMM standard errors of the estimate are only robust to heteroscedasticity, i.e. variance differs by observation.
Since we using time series for our estimation, there is a strong chance that the errors, 
or better moment conditions, are correlated over time. In this case, we can use a HAC robust standard error, 
that is robust to heteroscedasticity as well as autocorrelation.

In the GMM version in statsmodels, we define these options through `weights_method='hac', wargs={'maxlag':4}`,
which uses the Newey, West standard errors based on 4 lags.

In [6]:
# We don't need Nelder-Mead in this case, we can use bfgs default directly
#res1_ = mod1.fit([1,-1], maxiter=0, inv_weights=w0inv, opt_method='nm')

res1_hac4_2s = mod1.fit([1, -1], maxiter=2, inv_weights=w0inv, weights_method='hac', wargs={'maxlag':4})
print('\n\n')
print(res1_hac4_2s.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

Optimization terminated successfully.
         Current function value: 0.000539
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.055484
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9



                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        13.26
Model:                 NonlinearIVGMM   Prob (Hansen J):               0.00132
Method:                           GMM                                         
Date:                Mon, 24 Jun 2019                                         
Time:                        12:14:24                                         
No. Observations:                 239                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------

## The moment equations and GMM - version 2

In version 2 of the moment equations, we use that the non-instrument part of the Euler Equation has the form `1 - f(x, params)`.
Our `moment_consumption2` defines only the `f(x, params)' part and we use a vector of ones for endog.

In [7]:
def moment_consumption2(params, exog):
    beta, gamma = params
    #endog, exog = args
    r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)
    
    # 2nd part of moment condition without instrument    
    predicted = beta * (1. + r_forw1) * np.power(c_forw1 / c, -gamma)
    return predicted

In [8]:
endog2 = np.ones(exog.shape[0])    
mod2 = gmm.NonlinearIVGMM(endog2, exog, instrument, moment_consumption2, k_moms=4)
w0inv = np.dot(instrument.T, instrument) / len(endog2)  
res2_hac4_2s = mod2.fit([1,-1], maxiter=2, inv_weights=w0inv, weights_method='hac', wargs={'maxlag':4})

Optimization terminated successfully.
         Current function value: 0.000539
         Iterations: 4
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.055484
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9


In [9]:
print(res2_hac4_2s.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        13.26
Model:                 NonlinearIVGMM   Prob (Hansen J):               0.00132
Method:                           GMM                                         
Date:                Mon, 24 Jun 2019                                         
Time:                        12:14:24                                         
No. Observations:                 239                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
discount       0.9213      0.014     67.264      0.000       0.894       0.948
CRRA          -4.1154      1.508     -2.730      0.006      -7.070      -1.160


## Compare

We should get the same estimates in both versions for speciying the moment conditions.

In [10]:
res1_hac4_2s.params

array([ 0.92128497, -4.11536227])

In [11]:
res2_hac4_2s.params

array([ 0.92128497, -4.11536227])

And the two are the same (up to differences numerical precision of the algorithms) :

In [12]:
res1_hac4_2s.params - res2_hac4_2s.params, np.max(np.abs(res1_hac4_2s.params - res2_hac4_2s.params))

(array([ 0.,  0.]), 0.0)

Stata manual has params `[0.9205, -4.2224]` and standard errors equal to `[0.0135, 1.4739]`. Stata doesn't center the moments by default. We can get something closer to the results of Stata by using `centered = False` as a weights argument:

In [13]:
res_ = mod2.fit([1,-1], maxiter=2, inv_weights=w0inv, weights_method='hac', 
                wargs={'maxlag':4, 'centered':False}, optim_args={'disp':0})
print (res_.params)
print (res_.bse)

[ 0.9204602  -4.22268245]
[ 0.01345568  1.47338949]


## Extra: Iterated GMM

As comparison we can also iterate the GMM estimation until convergence.

The results look strange, especially with HAC robust weighting matrix. 
In both cases the estimate for the discount factor is large and close to one, however 
the preference parameter for relative risk aversion is 0.08 and -6.03 respectively.

This needs verification or an explanation. 
I have a test case for iterated GMM with heteroscedasticity robust standard errors 
where statsmodels and Stata agree at several decimals, so there might be something
problematic with this dataset.

**with HAC standard errors**

In [14]:
res2_hac_i = mod2.fit([1,-1], maxiter=100, inv_weights=w0inv, 
                      weights_method='hac', wargs={'maxlag':4}, 
                      optim_args={'disp':0})

In [15]:
#print(res2_hac4_i.summary(yname='Euler Eq', xname=['discount', 'CRRA']))
res2_hac_i.params

array([ 0.95862044,  0.07853092])

**with HC standard errors**

In [16]:
res2_i = mod2.fit([1,-1], maxiter=100, inv_weights=w0inv, optim_args={'disp':0})

In [17]:
#print(res2_i.summary(yname='Euler Eq', xname=['discount', 'CRRA']))
print (res2_i.params)

[ 0.90336824 -6.02649691]


## Extra: Problems with Optimization

When I ran several different cases for this example, I sometimes got convergence to different results. 
Although the objective function is quadratic in the moments, the momend conditions themselves can have 
more complicated non-linearities, that could be the cause for local minima.

In the above examples I used the function for the moment conditions as an argument to the class.
As alternative it is also possible to directly subclass a GMM class. This has the advantage
that we can override specific methods. For example, currently it is not possible to give a
analytical derivative for the moment conditions, but it can be added by subclassing and 
replacing the generic forward difference method `gradient_momcond`.
